In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from tqdm import tqdm
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager,rc
import matplotlib
%matplotlib inline

In [40]:
stocks = pd.read_csv('./trade_train_set/stocks.csv')
trade_train = pd.read_csv('./trade_train_set/trade_train.csv')

매수가격 중앙값과 시장가랑 얼마나 괴리가 큰지도 변수로 추가

각 그룹별 예측

각 월별 top3 종목들 쭉 뽑아서 총 몇종목 있는지 파악하기

In [41]:
print('stocks shape: {}'.format(stocks.shape))
print('\n')
print('trade train shape: {}'.format(trade_train.shape))
print('\n')
print('Number of groups: {}'.format(len(trade_train['그룹번호'].unique())))

stocks shape: (286061, 15)


trade train shape: (30200, 13)


Number of groups: 48


In [42]:
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

#폰트 경로
font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"
 
#폰트 이름 얻어오기
font_name = font_manager.FontProperties(fname=font_path).get_name()
 
#font 설정
matplotlib.rc('font',family=font_name)


In [43]:
stocks = stocks.drop(['index'], axis=1)
trade_train = trade_train.drop(['Unnamed: 0'], axis=1)

trade_train[(trade_train['그룹번호'] == 'MAD01') & (trade_train['종목번호'] == 'A000020')]

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값


거래대금 기준이 아니라 매수고객수 기준!!!

In [44]:
def num_one_history(df, grp):
    data = df[df['그룹번호'] == grp][['기준년월', '종목번호']]
    data['대분류'] = 0
    data['중분류'] = 0
    data['소분류'] = 0
    
    for i in range(data.shape[0]):
        data.iloc[i,2] = list(stocks[stocks['종목번호'] == list(data['종목번호'])[i]]['표준산업구분코드_대분류'].unique())[0]
        data.iloc[i,3] = list(stocks[stocks['종목번호'] == list(data['종목번호'])[i]]['표준산업구분코드_중분류'].unique())[0]
        data.iloc[i,4] = list(stocks[stocks['종목번호'] == list(data['종목번호'])[i]]['표준산업구분코드_소분류'].unique())[0]
    return data

In [45]:
adj = trade_train.sort_values(by=['기준년월', '그룹번호', '매수고객수'], ascending=[True, True, False]).groupby(['기준년월', '그룹번호']).head(3)

trade_train[trade_train['그룹번호'] == 'MAD07']

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
342,201907,MAD07,86,A009150,Y,Y,4,3,45,33,91017,90500
1854,201908,MAD07,86,A093370,Y,Y,3,3,2539,2718,8956,10234
3262,201909,MAD07,86,A009150,Y,Y,3,6,11,64,97500,101377
4680,201910,MAD07,86,A215600,Y,Y,3,3,738,975,10450,13650
6340,201911,MAD07,86,A005930,Y,Y,5,4,60,64,52000,52200
...,...,...,...,...,...,...,...,...,...,...,...,...
26291,202006,MAD07,86,A090430,Y,Y,3,5,14,35,162500,175500
26292,202006,MAD07,86,A090460,Y,N,3,0,40,0,20100,-1
26293,202006,MAD07,86,A091990,Y,Y,3,4,78,121,93480,93144
26294,202006,MAD07,86,A096770,Y,Y,16,19,79,124,125500,127500


In [46]:
num_one_history(adj, 'MAD07')

,기준년월,종목번호,대분류,중분류,소분류
342,201907,A009150,제조업,통신장비 제조업,전자부품 제조업
1854,201908,A093370,제조업,화학물질 및 화학제품 제조업;,기초 화학물질 제조업
3262,201909,A009150,제조업,통신장비 제조업,전자부품 제조업
4680,201910,A215600,전문· 과학 및 기술 서비스업,연구개발업,자연과학 및 공학 연구개발업
6340,201911,A005930,제조업,통신장비 제조업,통신 및 방송 장비 제조업
6341,201911,A010060,제조업,화학물질 및 화학제품 제조업;,기초 화학물질 제조업
6343,201911,A068270,제조업,의료용 물질 및 의약품 제조업,기초 의약물질 및 생물학적 제제 제조업
8060,201912,A005930,제조업,통신장비 제조업,통신 및 방송 장비 제조업
8059,201912,A000660,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업
9601,202001,A005930,제조업,통신장비 제조업,통신 및 방송 장비 제조업


In [47]:
def freq(df):
    big = pd.DataFrame(df['대분류'].value_counts().nlargest(3)).reset_index(drop=False)
    big.columns = ['대분류', '카운트']
    mid = pd.DataFrame(df['중분류'].value_counts().nlargest(3)).reset_index(drop=False)
    mid.columns = ['중분류', '카운트']
    small = pd.DataFrame(df['소분류'].value_counts().nlargest(3)).reset_index(drop=False)
    small.columns = ['소분류', '카운트']

    
    print(big)
    print('\n')
    print(mid)
    print('\n')
    print(small)

In [48]:
freq(num_one_history(adj, 'MAD14'))

                     대분류  카운트
0                    제조업   24
1  전기· 가스· 증기 및 공기조절 공급업   12


                     중분류  카운트
0               통신장비 제조업   18
1  전기· 가스· 증기 및 공기조절 공급업   12
2         자동차 및 트레일러 제조업    4


                 소분류  카운트
0     통신 및 방송 장비 제조업   16
1                전기업   12
2  자동차용 엔진 및 자동차 제조업    4


1,2: 통신장비 3&8&22: 의약품 7: 2019년 7,8,9월 한종목씩만 구입 10: 여러종목 구매 12: 금융 16:유류 26:다양 30&32&42&48: 제조업중 다양 34:거래량 낮음 35:반도체 39:전기업 40: 거래량 낮음 

In [49]:
adj = trade_train.sort_values(by=['기준년월', '그룹번호', '매수고객수'], ascending=[True, True, False]).groupby(['기준년월', '그룹번호']).head(3)

adj

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
10,201907,MAD01,288,A066570,Y,Y,15,9,60,223,69900,76600
8,201907,MAD01,288,A009150,Y,Y,14,10,76,83,94211,94900
4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850
14,201907,MAD02,112,A042670,Y,Y,4,4,872,2340,5900,6025
12,201907,MAD02,112,A006800,Y,Y,3,3,1280,6639,7760,7895
...,...,...,...,...,...,...,...,...,...,...,...,...
30131,202006,MAD47,244,A017670,Y,Y,46,21,27,42,208000,221000
30123,202006,MAD47,244,A005935,Y,Y,37,18,77,100,46311,46752
30178,202006,MAD48,163,A035420,Y,Y,26,27,52,51,242500,240000
30179,202006,MAD48,163,A035720,Y,Y,24,24,18,23,267500,271500


# 각 월별 시장 전체 top3

In [50]:
aa = trade_train[['기준년월', '종목번호', '매수고객수']].groupby(['기준년월','종목번호']).sum().sort_values(
    by = ['기준년월','매수고객수'], ascending=[True, False]).reset_index(drop=False)

aa.groupby(['기준년월']).head(3)

,기준년월,종목번호,매수고객수
0,201907,A005930,487
1,201907,A093370,339
2,201907,A009150,319
376,201908,A005930,736
377,201908,A215600,437
378,201908,A000660,257
726,201909,A005930,576
727,201909,A000660,283
728,201909,A084990,185
1068,201910,A215600,686


# 각 월별 top3 구성 종목들

In [51]:
a = trade_train[['기준년월', '그룹번호', '종목번호', '매수고객수']].groupby(['기준년월', '그룹번호']).head(3)

a[a['기준년월'] == 202005]

,기준년월,그룹번호,종목번호,매수고객수
21288,202005,MAD01,A000210,3
21289,202005,MAD01,A000660,50
21290,202005,MAD01,A000990,6
21323,202005,MAD02,A000120,0
21324,202005,MAD02,A000660,8
...,...,...,...,...
25221,202005,MAD47,A000660,20
25222,202005,MAD47,A001740,0
25248,202005,MAD48,A000270,3
25249,202005,MAD48,A000660,8


그룹내고객수, 매수여부 열들 삭제, 데이터는 최대한 많이 가져가서 합치기

데이터하나로 합치기~~~ 가격 = (거래금액 / 거래량)의 월별평균

stocks 월별 하나, 합친거 하나 총 두개의 데이터셋

# 하나의 데이터셋으로 합치기

In [52]:
import math

for i in range(len(stocks['기준일자'])):
    a = stocks['기준일자'][i] / 100
    a = math.floor(a)
    stocks['기준일자'][i] = a

<ipython-input-52-e04f7bdb8140>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks['기준일자'][i] = a


In [53]:
supple_dat = stocks.groupby(['기준일자', '종목번호']).sum().reset_index(drop=False)

supple_dat['거래금액_만원단위'] = supple_dat['거래금액_만원단위'] * 10000

supple_dat['평균가격'] = supple_dat['거래금액_만원단위'] / supple_dat['거래량']

supple_dat

supple_dat.to_csv('./supple_dat.csv')

supple_dat.head()

trade_train.head()

In [ ]:
from tqdm import tqdm
dat = trade_train.copy()
dat['평균가격'] = 0

for i in tqdm(range(dat.shape[0])):
    for j in range(supple_dat.shape[0]):
        if (dat['기준년월'][i] == supple_dat['기준일자'][j]) & (dat['종목번호'][i] == supple_dat['종목번호'][j]):
            dat['평균가격'][i] = supple_dat['평균가격'][j]

In [14]:
supple_dat = pd.read_csv('./supple_dat.csv')
supple_dat = supple_dat.drop(['Unnamed: 0'], axis=1)

dat = pd.read_csv('./dat.csv')
dat = dat.drop(['Unnamed: 0'], axis=1)

In [15]:
dat[dat['평균가격'] == 0].shape

(10, 13)

In [16]:
cnt = 0
for i in trade_train['종목번호'].unique():
    if i not in stocks['종목번호'].unique():
        cnt += 1

cnt

1

In [17]:
dat

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값,평균가격
0,201907,MAD01,288,A000660,Y,Y,7,17,19,234,74800,78500,74563
1,201907,MAD01,288,A001820,Y,Y,3,4,533,154,47385,46700,48992
2,201907,MAD01,288,A004020,Y,Y,3,3,409,528,41411,40000,40211
3,201907,MAD01,288,A005380,Y,Y,4,3,323,451,140602,140628,134906
4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850,46248
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30195,202006,MAD48,163,A207940,Y,Y,3,4,3,3,835000,797000,747104
30196,202006,MAD48,163,A272210,Y,Y,3,3,90,42,9900,8870,9315
30197,202006,MAD48,163,A285130,Y,Y,6,4,271,162,109100,98000,101090
30198,202006,MAD48,163,A316140,Y,Y,5,4,386,735,10050,9920,9400


In [59]:
supple_dat

,기준일자,종목번호,종목시가,종목고가,종목저가,종목종가,거래량,거래금액_만원단위,평균가격
0,201907,A000020,9564.347826,9644.347826,9374.782609,9501.739130,5.205826e+04,4.908391e+04,9484.943496
1,201907,A000080,21660.869565,21910.869565,21352.173913,21641.304348,3.887390e+05,8.437530e+05,21627.759147
2,201907,A000100,233152.173913,235717.391304,229500.000000,231500.000000,6.275791e+04,1.509332e+06,232138.050687
3,201907,A000120,135434.782609,137065.217391,133565.217391,135434.782609,4.299757e+04,5.829285e+05,135404.079751
4,201907,A000140,10033.913043,10545.217391,9701.304348,10004.782609,1.102576e+06,1.266896e+06,10094.386081
...,...,...,...,...,...,...,...,...,...
13907,202007,A336370,35375.000000,36365.000000,34590.000000,35545.000000,5.792286e+05,2.097056e+06,35504.702008
13908,202007,A33637K,18577.500000,19455.000000,17955.000000,18517.500000,1.946282e+05,3.816172e+05,18713.615473
13909,202007,A33637L,37472.500000,39175.000000,36402.500000,37492.500000,5.336430e+04,2.085001e+05,37770.182855
13910,202007,A344820,33242.500000,33770.000000,32577.500000,33127.500000,1.085001e+05,3.627362e+05,33102.449273


In [60]:
stocks.columns

Index(['기준일자', '종목번호', '종목명', '20년7월TOP3대상여부', '시장구분', '표준산업구분코드_대분류',
       '표준산업구분코드_중분류', '표준산업구분코드_소분류', '종목시가', '종목고가', '종목저가', '종목종가', '거래량',
       '거래금액_만원단위'],
      dtype='object')

In [ ]:
dat['대분류'] = np.empty(dat.shape[0])
dat['중분류'] = np.empty(dat.shape[0])
dat['소분류'] = np.empty(dat.shape[0])

for i in tqdm(range(dat.shape[0])):
    for j in range(stocks.shape[0]):
        if dat['종목번호'][i] == stocks['종목번호'][j]:
            dat['대분류'][i] == stocks['표준산업구분코드_대분류'][j]
            dat['중분류'][i] == stocks['표준산업구분코드_중분류'][j]
            dat['소분류'][i] == stocks['표준산업구분코드_소분류'][j]

  0%|          | 7/30200 [00:14<18:05:00,  2.16s/it]